# Join Statements - Lab

## Introduction

In this lab, you'll practice your knowledge of `JOIN` statements, using various types of joins and various methods for specifying the links between them.

## Objectives

You will be able to:
* Write SQL queries that make use of various types of joins
* Compare and contrast the various types of joins
* Discuss how primary and foreign keys are used in SQL
* Decide and perform whichever type of join is best for retrieving desired data

## CRM ERD

In this lab, you'll use the same customer relationship management (CRM) database that you saw from the previous lesson.
<img src='images/Database-Schema.png' width="600">

## Connecting to the Database
Import the necessary packages and connect to the database `'data.sqlite'`.

In [1]:
# Your code here
import pandas as pd
import sqlite3

conn = sqlite3.connect('data.sqlite')
cur = conn.cursor()

## Select the names of all employees in Boston 

Hint: join the employees and offices tables. Select the first and last name.

In [10]:
# Your code here
q = """
    SELECT lastName, firstName
    FROM employees AS e
    JOIN offices AS o
        ON e.officeCode = o.officeCode
    WHERE o.city = 'Boston' 
    ;"""
pd.read_sql(q, conn)

,lastName,firstName
0,Firrelli,Julie
1,Patterson,Steve


## Are there any offices that have zero employees?
Hint: Combine the employees and offices tables and use a group by. Select the office code, city, and number of employees.

In [12]:
# Your code here
q = """
    SELECT
        o.officeCode,
        o.city,
        COUNT(e.employeeNumber) AS num_employees
    FROM offices AS o
    LEFT JOIN employees as e
        USING(officeCode)
    GROUP BY officeCode
    HAVING num_employees = 0
    ;
    """
pd.read_sql(q, conn)

,officeCode,city,num_employees
0,27,Boston,0


## Write 3 questions of your own and answer them

In [17]:
# Answers will vary

# Example question: 
"""
How many customers are there per office?
"""
q = """
        SELECT o.city,
               COUNT(c.customerNumber)
        FROM offices AS o
        JOIN employees AS e
             USING(officeCode)
        JOIN customers AS c
             ON e.employeeNumber = c.salesRepEmployeeNumber
        GROUP BY officeCode
        ;
        """
pd.read_sql(q, conn)

,city,COUNT(c.customerNumber)
0,San Francisco,12
1,Boston,12
2,NYC,15
3,Paris,29
4,Tokyo,5
5,Sydney,10
6,London,17


In [49]:
"""
Question 1: How many customers for each employee?
"""
q1 = """
    SELECT COUNT(c.customerNumber) AS totalCustomers,
           e.lastName, e.firstName
    FROM customers AS c
    JOIN employees AS e
        ON c.salesRepEmployeeNumber = e.employeeNumber
    GROUP BY e.employeeNumber
    ORDER BY totalCustomers DESC
    ;"""
pd.read_sql(q1, conn)

,totalCustomers,lastName,firstName
0,10,Castillo,Pamela
1,9,Jones,Barry
2,8,Bott,Larry
3,8,Vanauf,George
4,7,Hernandez,Gerard
5,7,Tseng,Foon Yue
6,6,Gerard,Martin
7,6,Bondur,Loui
8,6,Patterson,Steve
9,6,Firrelli,Julie


In [30]:
"""
Question 2: How many employees per office?
"""

# Your code here
q2 = """
    SELECT COUNT(e.employeeNumber) AS num_employee, 
    o.city
    FROM employees AS e
    JOIN offices as o
        USING(officeCode)
    GROUP BY o.city
    ;"""
pd.read_sql(q2, conn)

,num_employee,city
0,2,Boston
1,2,London
2,2,NYC
3,5,Paris
4,6,San Francisco
5,4,Sydney
6,2,Tokyo


In [44]:
"""
Question 3: What are the top 5 most loyal customers?
"""

# Your code here
q3 = """
     SELECT COUNT(o.customerNumber) AS totalOrders, 
     c.customerName
     FROM customers AS c
     JOIN orders AS o
         ON c.customerNumber = o.customerNumber
     GROUP BY c.customerName
     ORDER BY totalOrders DESC
     LIMIT 5
     ;"""
pd.read_sql(q3, conn)

,totalOrders,customerName
0,26,Euro+ Shopping Channel
1,17,Mini Gifts Distributors Ltd.
2,5,Reims Collectables
3,5,"Dragon Souveniers, Ltd."
4,5,"Down Under Souveniers, Inc"


## Level Up 1: Display the names of every individual product that each employee has sold

Hint: You will need to use multiple `JOIN` clauses to connect all the way from employee names to product names.

In [54]:
# Your code here
q_up1 = """
        SELECT e.firstName, e.lastName, p.productName
        FROM employees AS e
        JOIN customers AS c
            ON e.employeeNumber = c.salesRepEmployeeNumber
        JOIN orders AS o
            USING(customerNumber)
        JOIN orderdetails AS od
            USING(orderNumber)
        JOIN products AS p
            USING(productCode)
        ;"""
pd.read_sql(q_up1, conn)

,firstName,lastName,productName
0,Leslie,Jennings,1958 Setra Bus
1,Leslie,Jennings,1940 Ford Pickup Truck
2,Leslie,Jennings,1939 Cadillac Limousine
3,Leslie,Jennings,1996 Peterbilt 379 Stake Bed with Outrigger
4,Leslie,Jennings,1968 Ford Mustang
...,...,...,...
2991,Martin,Gerard,1954 Greyhound Scenicruiser
2992,Martin,Gerard,1950's Chicago Surface Lines Streetcar
2993,Martin,Gerard,Diamond T620 Semi-Skirted Tanker
2994,Martin,Gerard,1911 Ford Town Car


## Level Up 2: Display the number of products each employee has sold

Alphabetize the results by employee last name.

Hint: Use the `quantityOrdered` column from `orderDetails`. Also, think about how to group the data when some employees might have the same first or last name.

In [59]:
# Your code here
q_up2 = """
        SELECT e.firstName, e.lastName, SUM(od.quantityOrdered) AS totalOrdered
        FROM employees AS e
        JOIN customers AS c
            ON e.employeeNumber = c.salesRepEmployeeNumber
        JOIN orders AS o
            USING(customerNumber)
        JOIN orderdetails AS od
            USING(orderNumber)
        JOIN products AS p
            USING(productCode)
        GROUP BY e.lastName, e.firstName
        ORDER BY e.lastName
        ;"""
pd.read_sql(q_up2, conn)

,firstName,lastName,totalOrdered
0,Loui,Bondur,6186
1,Larry,Bott,8205
2,Pamela,Castillo,9290
3,Julie,Firrelli,4227
4,Andy,Fixter,6246
5,Martin,Gerard,4180
6,Gerard,Hernandez,14231
7,Leslie,Jennings,11854
8,Barry,Jones,7486
9,Peter,Marsh,6632


## Level Up 3: Display the names employees who have sold more than 200 different products

Hint: this is different from the previous question because the quantity sold doesn't matter, only the number of different products

In [61]:
# Your code here
q_up3 = """
SELECT firstName, lastName, COUNT(productCode) as different_products_sold
FROM employees AS e
JOIN customers AS c
    ON e.employeeNumber = c.salesRepEmployeeNumber
JOIN orders
    USING(customerNumber)
JOIN orderdetails
    USING(orderNumber)
GROUP BY firstName, lastName
HAVING different_products_sold > 200
ORDER BY lastName
;
"""
pd.read_sql(q_up3, conn)

,firstName,lastName,different_products_sold
0,Larry,Bott,236
1,Pamela,Castillo,272
2,Gerard,Hernandez,396
3,Leslie,Jennings,331
4,Barry,Jones,220
5,George,Vanauf,211


## Summary

Congrats! You practiced using join statements and leveraged your foreign keys knowledge!